In [1]:
import numpy as np

In [34]:
def rungekutta4(f, x0, t):
    num_of_nodes = np.shape(t)[0]
    x = np.zeros((num_of_nodes, np.shape(x0)[0]))
    x[0] = x0
    print(num_of_nodes)
    for i in range(num_of_nodes - 1):
        print("x[i]: ", x[i])
        h = t[i+1] - t[i]
        k1 = f(x[i], t[i])
        k2 = f(x[i] + k1 * h / 2., t[i] + h / 2.)
        k3 = f(x[i] + k2 * h / 2., t[i] + h / 2.)
        k4 = f(x[i] + k3 * h, t[i] + h)
        x[i+1] = x[i] + (h / 6.) * (k1 + 2*k2 + 2*k3 + k4)
        print(i, h)
        print(k1, k2, k3, k4)
    return x


In [40]:
def calc_tau(f, alpha):
    if np.abs(np.max(f)) == 0:
        return 1e-16
    else:
        return 1e-2 * np.min(np.abs(alpha)) / np.abs(np.max(f))

def rungekutta4_t(f, x0, t_max):
    x = np.zeros((1, np.shape(x0)[0]))
    x[0] = x0
    t = 0
    while t < t_max:
        x_curr = x[-1]
        k1 = f(x_curr, t)

        h = calc_tau(k1, x_curr)
        print(t, h)
        k2 = f(x_curr + k1 * h / 2., t + h / 2.)
        k3 = f(x_curr + k2 * h / 2., t + h / 2.)
        k4 = f(x_curr + k3 * h, t + h)
        x_next = x_curr + (h / 6.) * (k1 + 2*k2 + 2*k3 + k4)

        x_next = x_next.reshape((1, 4))
        x = np.concatenate((x, x_next), axis=0)
        t = t+h

    return x

In [4]:
def calc_n_t(t, R_0=100, u=1e6, n_0=1e17):
    n = n_0 * (R_0 / (R_0 + u*t))**3
#     print(n)
    return n

In [5]:
def calc_T_t(t, R_0=100, u=1e6, T_0=2):
    T = T_0 * (R_0 / (R_0 + u*t))**2
#     print(T)
    return T

In [6]:
def calc_k_1(t, A=6.06e21, I_1=5.98):
    T = calc_T_t(t)
    n = calc_n_t(t)
    k1 = A / n * T **(3/2) * np.exp(-I_1 / T)
    return k1

def calc_k_2(t, A=6.06e21, I_2=18.83):
    T = calc_T_t(t)
    n = calc_n_t(t)
    k2 = A / n * T **(3/2) * np.exp(-I_2 / T)
    return k2
    
def calc_k_3(t, A=6.06e21, I_3=28.4):
    T = calc_T_t(t)
    n = calc_n_t(t)
    k3 = A / n * T **(3/2) * np.exp(-I_3 / T)
    return k3

In [15]:
def calc_j_1(t):
    T = calc_T_t(t)
    n = calc_n_t(t)
#     print("T^4.5", pow(T, 4.5))
    return 8.75e-27 * n**2/ T **(9/2)

def calc_j_2(t, A=6.06e21, I_2=18.83):
    T = calc_T_t(t)
    n = calc_n_t(t)
    return 7.0e-26 * n**2/ T **(9/2)
    
def calc_j_3(t, A=6.06e21, I_3=28.4):
    T = calc_T_t(t)
    n = calc_n_t(t)
    return 2.36e-25 * n**2/ T **(9/2)

In [16]:
def calc_func(alpha_vector, t):
    
    func = np.zeros_like(alpha_vector)
    
    alpha_0 = alpha_vector[0]
    alpha_1 = alpha_vector[1]
    alpha_2 = alpha_vector[2]
    alpha_3 = alpha_vector[3]
    
    alpha = alpha_1 + 2 * alpha_2 + 3 * alpha_3
    
    j_1 = calc_j_1(t)
    j_2 = calc_j_2(t)
    j_3 = calc_j_3(t)
    
    k_1 = calc_k_1(t)
    k_2 = calc_k_2(t)
    k_3 = calc_k_3(t)
    
#     print(t)
#     print("j:\n", j_1, j_2, j_3)
#     print("k:\n", k_1, k_2, k_3)
#     print("alpha", alpha)
#     print("diff", alpha_0 * k_1 - alpha * alpha_1)
    func[0] = - alpha * j_1 * (alpha_0 * k_1 - alpha * alpha_1)
    func[1] = alpha * j_1 * (alpha_0 * k_1 - alpha * alpha_1) - alpha * j_2 * (alpha_1 * k_2 - alpha * alpha_2)
    func[2] = alpha * j_2 * (alpha_1 * k_2 - alpha * alpha_2) - alpha * j_3 * (alpha_2 * k_3 - alpha * alpha_3)
    func[3] = alpha * j_3 * (alpha_2 * k_3 - alpha * alpha_3)

#     func[0] = - 1 * j_1 * (alpha_0 * k_1 - alpha * alpha_1)
#     func[1] = 1 * j_1 * (alpha_0 * k_1 - alpha * alpha_1) - 1 * j_2 * (alpha_1 * k_2 - alpha * alpha_2)
#     func[2] = 1 * j_2 * (alpha_1 * k_2 - alpha * alpha_2) - 1 * j_3 * (alpha_2 * k_3 - alpha * alpha_3)
#     func[3] = 1 * j_3 * (alpha_2 * k_3 - alpha * alpha_3)

#     print("func: \n", func)
#     print(np.sum(alpha_vector))
    assert func[0] == func[0]
    return func

In [41]:
# t = np.linspace(0, 1e-14, int(1e12))
alpha_0 = np.array([8621.424084890414, -8634.437408398891, 14.01351351168498, -0.00019000320715256778])
# alpha_0 = np.array([2.59333106e-05, 0.11553579, 0.83412872, 0.05030956])

sol = rungekutta4_t(calc_func, alpha_0, 1e-2)
# sol = rungekutta4(calc_func, alpha_0, t)

0 1e-16
1e-16 1.2566588635886707e-13
1.2576588635886706e-13 1.6656850839482008e-21
1.2576588802455214e-13 1.6657107866353826e-21
1.2576588969026291e-13 1.6657364896447193e-21
1.2576589135599941e-13 1.6657621929764536e-21
1.257658930217616e-13 1.6657878966306758e-21
1.257658946875495e-13 1.6658136006073668e-21
1.257658963533631e-13 1.6658393049064961e-21
1.257658980192024e-13 1.6658650095281099e-21
1.2576589968506742e-13 1.6658907144721895e-21
1.2576590135095814e-13 1.665916419739065e-21
1.2576590301687455e-13 1.6659421253280507e-21
1.2576590468281668e-13 1.6659678312395534e-21
1.257659063487845e-13 1.6659935374735316e-21
1.2576590801477804e-13 1.6660192440299992e-21
1.2576590968079729e-13 1.6660449509089486e-21
1.2576591134684224e-13 1.6660706581103588e-21
1.2576591301291288e-13 1.6660963656342998e-21
1.2576591467900925e-13 1.6661220734807185e-21
1.2576591634513132e-13 1.666147781649782e-21
1.2576591801127911e-13 1.6661734901411975e-21
1.257659196774526e-13 1.6661991989550884e-21
1.257

1.2576635877456603e-13 1.6729718216240385e-21
1.2576636044753786e-13 1.6729976156768897e-21
1.2576636212053546e-13 1.6730234100529754e-21
1.2576636379355886e-13 1.67304920475231e-21
1.2576636546660807e-13 1.6730749997749273e-21
1.2576636713968307e-13 1.6731007951207766e-21
1.2576636881278386e-13 1.6731265907899039e-21
1.2576637048591046e-13 1.6731523867824219e-21
1.2576637215906286e-13 1.6731781830980692e-21
1.2576637383224105e-13 1.6732039797369697e-21
1.2576637550544502e-13 1.6732297766991472e-21
1.257663771786748e-13 1.6732555739846159e-21
1.2576637885193036e-13 1.6732813715933237e-21
1.2576638052521172e-13 1.6733071695253386e-21
1.257663821985189e-13 1.6733329677806202e-21
1.2576638387185185e-13 1.673358766359214e-21
1.2576638554521062e-13 1.6733845652612338e-21
1.2576638721859518e-13 1.6734103644864052e-21
1.2576638889200554e-13 1.6734361640348651e-21
1.257663905654417e-13 1.6734619639066044e-21
1.2576639223890366e-13 1.6734877641016473e-21
1.2576639391239141e-13 1.673513564619985

1.2576690216895254e-13 1.6813460248981937e-21
1.2576690385029857e-13 1.6813719238312783e-21
1.257669055316705e-13 1.6813978230885392e-21
1.257669072130683e-13 1.6814237226699473e-21
1.2576690889449204e-13 1.6814496225755475e-21
1.2576691057594166e-13 1.6814755228056532e-21
1.2576691225741718e-13 1.6815014233595594e-21
1.257669139389186e-13 1.681527324237688e-21
1.2576691562044593e-13 1.6815532254399866e-21
1.2576691730199915e-13 1.6815791269664692e-21
1.2576691898357827e-13 1.6816050288171175e-21
1.257669206651833e-13 1.6816309309919762e-21
1.2576692234681422e-13 1.6816568334910484e-21
1.2576692402847104e-13 1.6816827363144488e-21
1.2576692571015379e-13 1.6817086394618915e-21
1.2576692739186243e-13 1.6817345429335337e-21
1.2576692907359698e-13 1.6817604467293794e-21
1.2576693075535743e-13 1.6817863508494406e-21
1.2576693243714377e-13 1.6818122552936877e-21
1.2576693411895602e-13 1.6818381600621777e-21
1.257669358007942e-13 1.6818640651548815e-21
1.2576693748265826e-13 1.681889970571768

1.2576741787094402e-13 1.6892862389841074e-21
1.2576741956023025e-13 1.6893122372440048e-21
1.2576742124954248e-13 1.689338235828934e-21
1.2576742293888072e-13 1.689364234739088e-21
1.2576742462824495e-13 1.6893902339740911e-21
1.2576742631763518e-13 1.6894162335341569e-21
1.2576742800705142e-13 1.6894422334192333e-21
1.2576742969649365e-13 1.6894682336293886e-21
1.257674313859619e-13 1.6894942341645711e-21
1.2576743307545613e-13 1.6895202350248164e-21
1.2576743476497637e-13 1.6895462362101257e-21
1.257674364545226e-13 1.689572237720626e-21
1.2576743814409485e-13 1.6895982395560526e-21
1.2576743983369309e-13 1.689624241716519e-21
1.2576744152331733e-13 1.6896502442020941e-21
1.2576744321296757e-13 1.6896762470127262e-21
1.2576744490264381e-13 1.6897022501484278e-21
1.2576744659234606e-13 1.6897282536092134e-21
1.257674482820743e-13 1.689754257395052e-21
1.2576744997182855e-13 1.6897802615061804e-21
1.2576745166160882e-13 1.6898062659422109e-21
1.257674533514151e-13 1.6898322707033037e-

1.257680005213912e-13 1.698248835284142e-21
1.2576800221964003e-13 1.6982749455240427e-21
1.2576800391791498e-13 1.6983010560899957e-21
1.2576800561621604e-13 1.698327166982116e-21
1.257680073145432e-13 1.6983532782001154e-21
1.2576800901289648e-13 1.6983793897441293e-21
1.2576801071127586e-13 1.6984055016141517e-21
1.2576801240968138e-13 1.6984316138102287e-21
1.25768014108113e-13 1.6984577263323182e-21
1.257680158065707e-13 1.6984838391804685e-21
1.2576801750505455e-13 1.6985099523546587e-21
1.257680192035645e-13 1.6985360658552054e-21
1.2576802090210057e-13 1.6985621796814956e-21
1.2576802260066274e-13 1.6985882938338235e-21
1.2576802429925105e-13 1.698614408312158e-21
1.2576802599786545e-13 1.6986405231166017e-21
1.2576802769650598e-13 1.69866663824708e-21
1.257680293951726e-13 1.6986927537036064e-21
1.2576803109386537e-13 1.698718869486206e-21
1.2576803279258423e-13 1.698744985595195e-21
1.2576803449132922e-13 1.6987711020299047e-21
1.257680361901003e-13 1.698797218790706e-21
1.25

...

1.2577773195465524e-13 1.8466709092133635e-21
1.2577773380132614e-13 1.8466988548737613e-21
1.2577773564802498e-13 1.8467268008767317e-21
1.257777374947518e-13 1.846754747222228e-21
1.2577773934150655e-13 1.8467826939104893e-21
1.2577774118828925e-13 1.8468106409411077e-21
1.257777430350999e-13 1.8468385883142862e-21
1.2577774488193849e-13 1.8468665360300643e-21
1.2577774672880502e-13 1.846894484088347e-21
1.257777485756995e-13 1.8469224324892482e-21
1.2577775042262193e-13 1.8469503812327086e-21
1.257777522695723e-13 1.8469783303187324e-21
1.2577775411655064e-13 1.8470062797475455e-21
1.2577775596355692e-13 1.8470342295187415e-21
1.2577775781059115e-13 1.847062179632534e-21
1.2577775965765332e-13 1.8470901300889255e-21
1.2577776150474346e-13 1.8471180808878722e-21
1.2577776335186155e-13 1.8471460320294486e-21
1.2577776519900758e-13 1.8471739835136472e-21
1.2577776704618156e-13 1.8472019353409314e-21
1.257777688933835e-13 1.847229887510321e-21
1.257777707406134e-13 1.8472578400223278e-2

1.2577823342598795e-13 1.854256725781245e-21
1.2577823528024468e-13 1.854284764388854e-21
1.2577823713452944e-13 1.8543128033401008e-21
1.2577823898884224e-13 1.854340842634514e-21
1.257782408431831e-13 1.854368882272371e-21
1.2577824269755198e-13 1.8543969222536747e-21
1.257782445519489e-13 1.854424962578391e-21
1.2577824640637385e-13 1.854453003246548e-21
1.2577824826082686e-13 1.8544810442580995e-21
1.257782501153079e-13 1.8545090856132855e-21
1.25778251969817e-13 1.854537127311734e-21
1.2577825382435412e-13 1.8545651693535986e-21
1.2577825567891928e-13 1.8545932117389443e-21
1.2577825753351249e-13 1.8546212544677118e-21
1.2577825938813374e-13 1.8546492975399546e-21
1.2577826124278304e-13 1.8546773409556126e-21
1.2577826309746038e-13 1.854705384714753e-21
1.2577826495216577e-13 1.8547334288179153e-21
1.257782668068992e-13 1.854761473263965e-21
1.2577826866166069e-13 1.8547895180534933e-21
1.257782705164502e-13 1.8548175631865143e-21
1.2577827237126778e-13 1.854845608662995e-21
1.257

1.2577874440412018e-13 1.8619802919247797e-21
1.2577874626610049e-13 1.86200842509148e-21
1.257787481281089e-13 1.862036558602653e-21
1.2577874999014548e-13 1.862064692457938e-21
1.2577875185221017e-13 1.8620928266575403e-21
1.25778753714303e-13 1.862120961201449e-21
1.2577875557642397e-13 1.8621490960896423e-21
1.2577875743857307e-13 1.862177231322162e-21
1.257787593007503e-13 1.8622053668990104e-21
1.2577876116295568e-13 1.8622335028201406e-21
1.2577876302518917e-13 1.8622616390858197e-21
1.2577876488745081e-13 1.862289775695611e-21
1.2577876674974058e-13 1.8623179126497307e-21
1.2577876861205849e-13 1.86234604994817e-21
1.2577877047440454e-13 1.8623741875909576e-21
1.2577877233677872e-13 1.8624023255780825e-21
1.2577877419918104e-13 1.8624304639095374e-21
1.2577877606161151e-13 1.8624586025858406e-21
1.257787779240701e-13 1.8624867416059866e-21
1.2577877978655684e-13 1.8625148809704847e-21
1.2577878164907173e-13 1.8625430206793364e-21
1.2577878351161476e-13 1.8625711607325204e-21
1.

1.25779283688586e-13 1.8701251180820056e-21
1.2577928555871112e-13 1.870153350883846e-21
1.2577928742886447e-13 1.8701815840308955e-21
1.2577928929904605e-13 1.870209817523561e-21
1.2577929116925588e-13 1.8702380513610613e-21
1.2577929303949393e-13 1.8702662855438305e-21
1.2577929490976022e-13 1.870294520071834e-21
1.2577929678005474e-13 1.870322754945061e-21
1.257792986503775e-13 1.8703509901635417e-21
1.257793005207285e-13 1.8703792257273025e-21
1.2577930239110773e-13 1.870407461636487e-21
1.2577930426151518e-13 1.870435697890757e-21
1.257793061319509e-13 1.870463934490267e-21
1.2577930800241482e-13 1.87049217143502e-21
1.25779309872907e-13 1.8705204087250324e-21
1.257793117434274e-13 1.8705486463603448e-21
1.2577931361397603e-13 1.8705768843408974e-21
1.2577931548455292e-13 1.8706051226667317e-21
1.2577931735515804e-13 1.8706333613380274e-21
1.257793192257914e-13 1.8706616003544452e-21
1.25779321096453e-13 1.8706898397161172e-21
1.2577932296714284e-13 1.8707180794230825e-21
1.257793

KeyboardInterrupt: 

In [138]:
import matplotlib.pyplot as plt

# plt.plot(t, sol[:,1])